In [ ]:
# import all the necessary libraries
import pandas as pd
import numpy as np
import altair as alt
import plotly.graph_objs as go

In [ ]:
# read and create data frames
df_car = pd.read_csv('car-assignments.csv')
df_cc = pd.read_csv('cc_data.csv', encoding = 'Windows-1252')
df_gps = pd.read_csv('gps.csv')
df_loyalty = pd.read_csv('loyalty_data.csv', encoding = 'Windows-1252')

### Having the clean data from Analysis portion:

In [ ]:
# Dropped missing values
new_df_car = df_car.dropna()

In [ ]:
# Converting the CarID float to int type
new_df_car['CarID'] = new_df_car['CarID'].astype(int)
new_df_car.info() # check if it worked

In [ ]:
# Change the timestamp type object to datetime
df_gps['Timestamp'] = pd.to_datetime(df_gps['Timestamp'])
df_gps.info() # check if it converted

In [ ]:
# Change the timestamp type object to datetime
df_cc['timestamp'] = pd.to_datetime(df_cc['timestamp'])
df_cc.info() # check if it converted

In [ ]:
# Change the timestamp type object to datetime
df_loyalty['timestamp'] = pd.to_datetime(df_loyalty['timestamp'])
df_loyalty.info() # check if it converted

### Merging with the credit card and loyalty purchases

In [ ]:
# deep copy the df of cc and loyalty
new_df_cc = df_cc.copy()
new_df_loyalty = df_loyalty.copy()

# Use strftime to convert the timestamp to year-month-day format
new_df_cc['timestamp'] = new_df_cc['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d'))

# Convert the timestamp column to a datetime object
new_df_cc['timestamp'] = pd.to_datetime(new_df_cc['timestamp'])

In [ ]:
# Remove duplicates and null values from the timestamp and location columns
new_df_cc = new_df_cc.drop_duplicates(subset=['timestamp', 'location']).dropna(subset=['timestamp', 'location'])
new_df_loyalty = new_df_loyalty.drop_duplicates(subset=['timestamp', 'location']).dropna(subset=['timestamp', 'location'])

# Perform a left join on the timestamp and location columns
combined_df = pd.merge(new_df_cc, new_df_loyalty, on=['timestamp', 'location'],
                       how='left', suffixes=('_credit', '_loyalty'))

In [ ]:
combined_df

In [ ]:
# Rename the timestamp column so easier to merge data
df_gps.rename(columns = {'Timestamp':'timestamp'}, inplace = True)

### Merging Datasets

In [ ]:
# Merge gps and cc data based on timestamp
cc = df_gps.merge(df_cc, how = 'inner', on = 'timestamp')

# Filter for ids less than or equal to 35
cc = cc[cc['id'] <= 35]
cc.head()

# Visualization 1: Bubble Chart

We will create two bubble charts. They will represent the purchase frequency from the credit card data set and compancy loyalty card data set.

In [ ]:
# Create bubble chart using credit card data
# Group by location and aggregate frequency of purchases
location_grouped = df_cc.groupby('location').agg({'price': 'count'}).reset_index()

# Create bubble chart
fig = go.Figure(data=go.Scatter(
    x=location_grouped['location'], 
    y=location_grouped['price'], 
    mode='markers',
    marker=dict(
        size=location_grouped['price'], # Set size based on frequency
        sizemode='diameter',
        sizeref= 0.9, # Set a smaller sizeref to make bubbles smaller
        color=location_grouped['price'], # Set color based on frequency
        colorbar=dict(title='Frequency'),
        colorscale='Viridis'
    ),
    text=location_grouped['location'] # Set location as label for each bubble
))

# Set layout
fig.update_layout(
    title='Frequency of Purchases by Location based on Credit Card Data',
    xaxis_title='Location',
    yaxis_title='Frequency'
)

# Show chart
fig.show()

In [ ]:
# Create bubble chart using loyalty data
# Group by location and aggregate frequency of purchases
loyalty_bubble = df_loyalty.groupby('location').agg({'price': 'count'}).reset_index()

# Create bubble chart
fig = go.Figure(data=go.Scatter(
    x = loyalty_bubble['location'], 
    y = loyalty_bubble['price'], 
    mode ='markers',
    marker = dict(
        size=loyalty_bubble['price'], # Set size based on frequency
        sizemode='diameter',
        sizeref=0.8,
        color= loyalty_bubble['price'], # Set color based on frequency
        colorbar=dict(title='Frequency'),
        colorscale='Viridis'
    ),
    text=loyalty_bubble['location'] # Set location as label for each bubble
))

# Set layout
fig.update_layout(
    title='Frequency of Purchases by Location based on Loyalty Card Data',
    xaxis_title='Location',
    yaxis_title='Frequency'
)

# Show chart
fig.show()

# Visualization 2: Heatmap

In [ ]:
# Generate the heatmap
heatmap = alt.Chart(cc).mark_rect().encode (
    x = alt.X('last4ccnum', bin=alt.Bin(maxbins=35)) , 
    y = alt.Y('id', bin=alt.Bin(maxbins=35)),
    color = 'count()',
    tooltip = 'last4ccnum'
).properties(
    title = 'Heatmap of Employee Credit Card Transaction Data',
    width = 600,
    height = 600
)
# Add a drop down meny to look at certain locations
# Create a dropdown menu for the locations
location_dropdown = alt.binding_select(options = cc['location'].unique().tolist(), name='Location')
id_select = alt.selection_single(fields=['location'], bind=location_dropdown, 
                                       init={'location': cc['location'].unique()[0]})

# Add the dropdown menu to the chart
heatmap = heatmap.add_selection(id_select).transform_filter(id_select)

# Display the chart
heatmap

##### The above heatmap is supposed to show the credit card transactions for each of the 35 employees given. The higher the count of records (the darker the rectangle), the more likely that credit card number belongs to the corresponding employee. The user can hover over the rectangles to see the credit card number, and the corresponding id number is on the left. We also added a drop down menu to change the location of purchase, so a different heatmap for each location.

# Visualization 3: Time-series of Kronos Kares Purchases

In [ ]:
# Create a new dataframe with the total purchase amount for each day and location
daily_data = combined_df.groupby([pd.Grouper(key='timestamp', freq='D'), 'location']).sum().reset_index()

# Create an interactive line chart for purchases using Altair
chart_cc = alt.Chart(daily_data).mark_line().encode(
    x='timestamp:T',
    y= alt.Y('price_credit:Q', axis=alt.Axis(format='$,.2f')),
    color='location:N'
).properties(
    title='Purchases Over Time by Location',
    width=800,
    height=400
)

chart_loyalty = alt.Chart(daily_data).mark_line().encode(
    x='timestamp:T',
    y= alt.Y('price_loyalty:Q',axis=alt.Axis(format='$,.2f')),
    color='location:N'
).properties(
    title='Purchases Over Time by Location',
    width=800,
    height=400
)

# Create a dropdown menu for the locations
location_dropdown = alt.binding_select(options=daily_data['location'].unique().tolist(), name='Location:')
location_select = alt.selection_single(fields=['location'], bind=location_dropdown,
                                       init={'location': daily_data['location'].unique()[0]})

# Add the dropdown menu to the chart
chart_cc = chart_cc.add_selection(location_select).transform_filter(location_select)
chart_loyalty = chart_loyalty.add_selection(location_select).transform_filter(location_select)

# Display the chart
chart_cc & chart_loyalty

In [ ]:
# Create a day column to filter by day
df_cc['day'] = df_cc['timestamp'].dt.date 

# Convert day column to string format
df_cc['day'] = df_cc['day'].astype(str)

# Create an interactive scatter plot with Altair
scatter_plot = alt.Chart(df_cc).mark_tick(size=20).encode(
    x=alt.X('hours(timestamp):T', axis=alt.Axis(title='Time of Day', format='%I:%M %p')),
    y=alt.Y('location:N', axis=alt.Axis(title='Location')),
    color=alt.Color('location:N', scale=alt.Scale(scheme='dark2')),
    tooltip=['location', 'timestamp', 'price'],
    opacity=alt.value(0.7),
).configure_tick(
    thickness=3,
    bandSize=10,
).properties(
    width=800,
    height=400,
    title='Transactions Over Time by Location'
).interactive()

# Create a dropdown menu for the days
day_dropdown = alt.binding_select(options=df_cc['day'].unique().astype(str).tolist(), name='Day:')
day_select = alt.selection_single(fields=['day'], bind=day_dropdown, init={'day': df_cc['day'].unique().astype(str)[0]})

# Add the dropdown menus to the scatter plot
scatter_plot = scatter_plot.add_selection(day_select).transform_filter(day_select)

# Display scatter plot
scatter_plot

# Visualization 5: Stacked/Regular Bar Chart of Purchases

In [ ]:
# Sort the id column
cc_sorted = cc.sort_values(by='id')

# Define selection filters
id_dropdown = alt.binding_select(options=list(cc_sorted['id'].unique()), name="Filter by ID: ")
id_select = alt.selection_single(fields=['id'], bind=id_dropdown, init={'id': cc_sorted['id'].unique()[0]})

location_select = alt.selection_multi(fields=['location'], bind='legend')

# Create stacked bar chart
base_id = alt.Chart(cc_sorted).mark_bar().encode(
    x='timestamp:T',
    y='sum(price)',
    color=alt.condition(
        location_select,
        alt.Color('location:N', legend=alt.Legend(title='Location')),
        alt.value('lightgray')
    ),
    tooltip=['timestamp', 'id:Q', 'location:N', 'last4ccnum:N', 'price:Q']
).properties(
    width=800,
    height=500,
    title='Purchase History Over Time'
).add_selection(
    location_select
).interactive()

# Add the dropdown menus to the base_id
base_id = base_id.add_selection(id_select).transform_filter(id_select)

# Display the chart
base_id

# Extra: Map of Vehicle Movements 

The goal is to map the movements of the company vehicles onto the provided ficitious map. So far we have plotted the gps coordinates by doing a moving average for every 4 hours of each day of the two week period. The user can see the layout for each employee by clicking on the color-coded bar graph.

In [ ]:
# Set the index of the data set as the timestamp (to manipulate later)
gps = df_gps.set_index('timestamp') 
gps.head()

In [ ]:
# Take the average gps coord for every day of the two week period
g = gps.groupby('id', as_index = False)
#g.head()

# Aggregate the 4 hour average of every day in the 2 week period
average_pos = g.resample('4H').mean()
average_pos.head()

In [ ]:
# Remove the rows with any missing values
pos = average_pos.dropna()
pos = pos[pos['id'] <= 35]
pos.tail() # 1828 row observations
# Now we have a data frame with the average latitude and longitude coordinates for each employee id

In [ ]:
# Turn the datetime index back to a column variable again
pos = pos.reset_index() # drop the index and move it to be a column
pos.head()

In [ ]:
# Drop the unnecessary column variable(s)
pos.drop('level_0', axis = 'columns')

In [ ]:
# Create the Visualization

# Create a clicker that will change the data shown on the scatter plot
click = alt.selection_multi(encodings=['color'])

# Create a bar chart where the user can select the employee id and see a path for each person
bar = alt.Chart(pos).mark_bar().encode(
    #x = 'count()',
    y = 'id:O',
    color = alt.condition(click, 'id:N', alt.value('lightgray'), legend = None)
).add_selection(
    click
)
# create the scatter plot
scatter = alt.Chart(pos).mark_circle(size = 40, opacity = 1).encode(
    x = alt.X('lat', scale = alt.Scale(domain = [36.04, 36.09])),
    y = alt.Y('long', scale = alt.Scale(domain = [24.84,24.92])),
    color = 'id:N',
    tooltip = ['timestamp']
).transform_filter(
    click
)

# join the data sets vertically
#bar
scatter | bar